## Xarray engine: hypercube with holes

This notebook demonstrates how to work with input data not forming a full hypercube. 

First, we get pressure level GRIB data with t, u and v variables, where u only contains the subset of the levels in t and v. This data forms a hypercube with holes.

In [1]:
import earthkit.data as ekd
ds_fl = ekd.from_source("sample", "tuv_pl_holes.grib")

tuv_pl_holes.grib:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

In [2]:
try:
    ds_fl.to_xarray()
except ValueError as e:
    import textwrap
    print(textwrap.fill(str(e), 100))

Variable "u" has inconsistent dimension "levelist" compared to other variables. Expected values: (6)
[300, 400, 500, 700, 850, 1000], got: (2) [500, 700]. Length mismatch: 6 != 2


We need to use the ``allow_holes=True`` option to make the conversion work.

In [3]:
ds = ds_fl.to_xarray(allow_holes=True)
ds

<xarray.Dataset> Size: 12kB
Dimensions:    (level: 6, latitude: 7, longitude: 12)
Coordinates:
  * level      (level) int64 48B 300 400 500 700 850 1000
  * latitude   (latitude) float64 56B 90.0 60.0 30.0 0.0 -30.0 -60.0 -90.0
  * longitude  (longitude) float64 96B 0.0 30.0 60.0 90.0 ... 270.0 300.0 330.0
Data variables:
    t          (level, latitude, longitude) float64 4kB ...
    u          (level, latitude, longitude) float64 4kB ...
    v          (level, latitude, longitude) float64 4kB ...
Attributes:
    class:        od
    stream:       oper
    levtype:      pl
    type:         an
    expver:       0001
    date:         20180801
    time:         1200
    domain:       g
    number:       0
    Conventions:  CF-1.8
    institution:  ECMWF

The resulting dataset works as expected but when we slice into the holes ``nans`` are returned.

In [4]:
ds.u.sel(level=300).values

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [5]:
ds.u.sel(level=[500,300]).values

array([[[  4.74000549,   4.74000549,   4.74000549,   4.74000549,
           4.74000549,   4.74000549,   4.74000549,   4.74000549,
           4.74000549,   4.74000549,   4.74000549,   4.74000549],
        [ 12.74000549,   4.74000549,   4.74000549,  28.74000549,
          12.74000549,  -3.25999451,   4.74000549,   4.74000549,
           4.74000549,  12.74000549,  20.74000549,   4.74000549],
        [ -3.25999451,   4.74000549,  -3.25999451,   4.74000549,
         -11.25999451,   4.74000549,  -3.25999451,  -3.25999451,
          -3.25999451,   4.74000549, -11.25999451,  12.74000549],
        [  4.74000549,  -3.25999451,  -3.25999451,  12.74000549,
          -3.25999451, -11.25999451, -11.25999451,  -3.25999451,
           4.74000549,  -3.25999451, -19.25999451,   4.74000549],
        [ 20.74000549,  36.74000549,  20.74000549,  12.74000549,
          20.74000549,  12.74000549,  12.74000549,  36.74000549,
          12.74000549,   4.74000549,  20.74000549,  12.74000549],
        [ 36.7400054